In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src')) # include top level package in python path

In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
.container { 
    width: 100% !important;
}
</style>
"""))

In [3]:
import torch
from torch import nn, Tensor
from model.fuzzy_layer import (
    FuzzySignedConjunction, 
    FuzzySignedDisjunction, 
    FuzzyUnsignedConjunction, 
    FuzzyUnsignedDisjunction, 
    FuzzyNumKeepup, 
    keepidx, 
    fuzzy_dropup
)
from model.fuzzy_logic import ProductLogic, MinimumLogic, LukasiewiczLogic, DrasticLogic, SchweizerSklarLogic
from model.fuzzy_layer import FuzzyParam
from model.bool_logic import BoolLogic
from cache import TestMetric, TrainingRegime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from util import shuffle

device = 'cuda'

logic = BoolLogic()
flogic = ProductLogic()

In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

trainset = datasets.MNIST(root='./_mnist', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.MNIST(root='./_mnist', train=False, download=True, transform=transforms.ToTensor())

trainloader = DataLoader(trainset, batch_size=32)
testloader = DataLoader(testset, batch_size=32)

In [5]:
class CorrectOutputMetric(TestMetric):
    def name(self):
        return "correct-output"
    
    def measure_model(self, model, it):
        correct_count = 0
        total_count = 0

        with torch.no_grad():
            for imgs, labels in it():
                preds = model(imgs.to(device))
                if hasattr(model.__class__, "zero_reg"):
                    model.zero_reg()
                pred_labels = preds.argmax(dim=-1)
                correct_count += (pred_labels == labels.to(device)).sum()
                total_count += labels.numel()

        return (correct_count / total_count).item()
    
class LossMetric(TestMetric):
    def name(self):
        return "test-loss"

    def measure_model(self, model, it):
        loss_fn = nn.CrossEntropyLoss()

        total_loss = 0
        total_count = 0
        
        with torch.no_grad():
            for imgs, labels in it():
                preds = model(imgs.to(device))
                if hasattr(model.__class__, "zero_reg"):
                    model.zero_reg()
                loss = loss_fn(preds, labels.to(device))
                total_loss += loss
                total_count += labels.numel()

        return (total_loss / total_count).item()

In [6]:
class EmbedRegMetric(TestMetric):
    def name(self):
        return "embed-reg"
    
    def measure_model(self, model, it):
        total_count = 0
        
        with torch.no_grad():
            for imgs, _ in it():
                model(imgs.to(device))
                total_count += imgs.size(0)

        reg = model.logic_reg()
        model.zero_reg()
            
        return (reg / total_count).item()

In [7]:
class ModelMNIST(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.Sigmoid(),
            nn.Linear(128, 64),
            nn.Sigmoid(),
            nn.Linear(64, 10),
            nn.Softmax(dim=-1),
        )
        
    def forward(self, input):
        return self.model(input)

In [8]:
class FuzzyMNIST(nn.Module):
    
    def __init__(self, logic, lin_channels, logic_layers=1, logic_width=64, keepn=None):
        super().__init__()
        self.lin_channels = lin_channels
        self.logic_width = logic_width
        self.logic_layers = logic_layers
        self.keepn = keepn
        
        fuzzy_intermed_layers = [(
            FuzzySignedDisjunction(logic_width, logic_width, logic=logic, keepn=keepn),
            FuzzySignedConjunction(logic_width, logic_width, logic=logic, keepn=keepn)
        ) for i in range(1, logic_layers)] 
        fuzzy_intermed_layers = [layer for tup in fuzzy_intermed_layers for layer in tup]
        
        lin = nn.Linear(28*28, self.lin_channels)
        
        self.model = nn.Sequential(
            nn.Flatten(),
            lin,
            nn.Sigmoid(),
            FuzzySignedConjunction(self.lin_channels, logic_width, logic=logic, keepn=keepn),
            *fuzzy_intermed_layers,
            FuzzySignedDisjunction(logic_width, 10, logic=logic, keepn=keepn),
            nn.Softmax(dim=-1),
        )
        
        torch.nn.init.xavier_uniform_(lin.weight)
        
    def forward(self, input):
        return self.model(input)

In [9]:
from model.embed_layer import EmbedDNF, EmbedEncode, EmbedDecode
from model.embed_logic import EmbedLogic

class EmbedMNIST(nn.Module):
    
    def __init__(self, lin_width, logic_layers=1, logic_width=64, embed_dims=3, reg=True):
        super().__init__()
        
        self.logic = EmbedLogic(embed_dims, calculate_reg=reg)
        self.lin_width = lin_width
        self.logic_width = logic_width
        self.logic_layers = logic_layers
        
        if logic_layers == 1:
            embed_widths = [(self.lin_width, logic_width, 10)]
        else:
            embed_widths = [
                (self.lin_width, logic_width, logic_width),
                *((logic_width, logic_width, logic_width) for _ in range(logic_layers - 2)),
                (logic_width, logic_width, 10),
            ]
            
        embed_layers = [EmbedDNF(shape=shape, logic=self.logic) for shape in embed_widths]
        
        self.lin_model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, self.lin_width),
            nn.ReLU(),
            nn.Linear(self.lin_width, self.lin_width * embed_dims),
        )
            
        self.embed_model = nn.Sequential(
            EmbedEncode(embed_dims=embed_dims),
            *embed_layers,
            EmbedDecode(logic=self.logic),
            nn.Softmax(dim=-1),
        )
            
    def forward(self, input):
        dims = self.logic.embed_dims
        a_0 = self.lin_model(input)
        z_0 = a_0.reshape(*a_0.shape[:-1], a_0.size(-1) // dims, dims)
        return self.embed_model(z_0)
    
    def logic_reg(self):
        return self.logic.logic_reg()
    
    def zero_reg(self):
        return self.logic.zero_reg()

In [10]:
class MNISTRegime(TrainingRegime):
    def __init__(self, lr=1e-2, no_runs=1):
        super().__init__("./mnist/", no_runs)
        
        self.lr = lr
        
        self.tests = [
            CorrectOutputMetric(),
            LossMetric(),
        ]
        
        self.trainloader = DataLoader(trainset, batch_size=128)
        self.testloader = DataLoader(testset, batch_size=128)
        
        self.optims = [None] * no_runs
        
    def get_optim(self, run_no):
        optim = self.optims[run_no - 1]
        if optim is None:
            model = self.get_loaded_model(run_no)
            optim = self.optims[run_no - 1] = (
                torch.optim.Adam(model.parameters(), lr=self.lr)
            )
        return optim
        
    def regime_str(self):
        return (
            "MNIST | ADAM, LR = %s | STANDARD MODEL"
            % ( 
                self.lr
                
            )
        ) 
        
    def regime_filename_elems(self):
        elems = [
            "mnist",
            str(self.lr),
            "standard",
        ]
        return elems
        
    def training_dataloader(self, run_no):
        return self.trainloader
    
    def testing_dataloader(self, run_no):
        return self.testloader
    
    def training_step(self, run_no, model):
        optim = self.get_optim(run_no)
        loss_fn = nn.CrossEntropyLoss()
        
        def step(tup):
            imgs, labels = tup
            preds = model(imgs.to(device))
            loss = loss_fn(preds, labels.to(device))
            
            optim.zero_grad(set_to_none=True)
            loss.backward()
            
            grads = torch.tensor(0.0)
            for param in model.parameters():
                with torch.no_grad():
                    print(param.grad.abs().sum().cpu(), param.grad.shape)
            raise Error
            
            optim.step()
            
        return step
        
    def new_model(self):
        return ModelMNIST().to(device)

In [11]:
class MNISTRealRegime(TrainingRegime):
    def __init__(self, logic, lin_channels=16, logic_width=64, logic_layers=1, keepn=32, lr=1e-2, exp=1, no_runs=1):
        super().__init__("./mnist/", no_runs)
        
        self.flogic = logic
        self.lin_channels = lin_channels
        self.logic_width = logic_width
        self.logic_layers = logic_layers
        self.keepn = keepn
        self.lr = lr
        self.exp = exp
        
        self.tests = [
            CorrectOutputMetric(),
            LossMetric(),
        ]
        
        self.trainloader = DataLoader(trainset, batch_size=32)
        self.testloader = DataLoader(testset, batch_size=32)
        
        self.optims = [None] * no_runs
        
    def get_logic_str(self):
        if isinstance(self.flogic, ProductLogic):
            return "product"
        elif isinstance(self.flogic, MinimumLogic):
            return "minimum"
        elif isinstance(self.flogic, LukasiewiczLogic):
            return "lukasiewicz"
        elif isinstance(self.flogic, DrasticLogic):
            return "drastic"
        elif isinstance(self.flogic, SchweizerSklarLogic):
            return "schweizer-sklar"
        else:
            return "fuzzy"
        
    def get_optim(self, run_no):
        optim = self.optims[run_no - 1]
        if optim is None:
            model = self.get_loaded_model(run_no)
            optim = self.optims[run_no - 1] = (
                torch.optim.Adam(model.parameters(), lr=self.lr)
            )
        return optim
        
    def regime_str(self):
        keep_str = "" if self.keepn is None else " | KEEP N = %s" % self.keepn
        return (
            "MNIST | ADAM, LR = %s | %s LOGIC | %s LINEAR CHANNELS | WIDTH %s LOGIC LAYER | %s LOGIC LAYERS | EXP %s%s"
            % ( 
                self.lr,
                self.get_logic_str().upper(), 
                self.lin_channels, 
                self.logic_width, 
                self.logic_layers, 
                self.exp,
                keep_str
                
            )
        ) 
        
    def regime_filename_elems(self):
        elems = [
            "mnist",
            str(self.lr),
            self.get_logic_str(),
            "%slc" % self.lin_channels,
            "%sw" % self.logic_width,
            "%slyrs" % self.logic_layers
        ]
        if self.exp != 1:
            elems.append("exp%s" % self.exp)
        if self.keepn is not None:
            elems.append("keep%s" % self.keepn)
        return elems
        
    def training_dataloader(self, run_no):
        return self.trainloader
    
    def testing_dataloader(self, run_no):
        return self.testloader
    
    def training_step(self, run_no, model):
        optim = self.get_optim(run_no)
        loss_fn = nn.CrossEntropyLoss()
        
        def step(tup):
            imgs, labels = tup
            preds = model(imgs.to(device))
            loss = loss_fn(preds, labels.to(device)) ** self.exp
            #losses.append(loss.item())

            optim.zero_grad(set_to_none=True)
            loss.backward()
            optim.step()
            
        return step
        
    def new_model(self):
        return EmbedMNIST(
            logic=self.flogic, 
            lin_channels=self.lin_channels, 
            logic_layers=self.logic_layers, 
            logic_width=self.logic_width,
        ).to(device)

In [ ]:
class MNISTRealMLPRegime(TrainingRegime):
    def __init__(self, logic, lin_channels=16, logic_width=64, logic_layers=1, keepn=32, lr=1e-2, exp=1, no_runs=1):
        super().__init__("./mnist/", no_runs)
        
        self.flogic = logic
        self.lin_channels = lin_channels
        self.logic_width = logic_width
        self.logic_layers = logic_layers
        self.keepn = keepn
        self.lr = lr
        self.exp = exp
        
        self.tests = [
            CorrectOutputMetric(),
            LossMetric(),
        ]
        
        self.trainloader = DataLoader(trainset, batch_size=32)
        self.testloader = DataLoader(testset, batch_size=32)
        
        self.optims = [None] * no_runs
        
    def get_logic_str(self):
        if isinstance(self.flogic, ProductLogic):
            return "product"
        elif isinstance(self.flogic, MinimumLogic):
            return "minimum"
        elif isinstance(self.flogic, LukasiewiczLogic):
            return "lukasiewicz"
        elif isinstance(self.flogic, DrasticLogic):
            return "drastic"
        elif isinstance(self.flogic, SchweizerSklarLogic):
            return "schweizer-sklar"
        else:
            return "fuzzy"
        
    def get_optim(self, run_no):
        optim = self.optims[run_no - 1]
        if optim is None:
            model = self.get_loaded_model(run_no)
            optim = self.optims[run_no - 1] = (
                torch.optim.Adam(model.parameters(), lr=self.lr)
            )
        return optim
        
    def regime_str(self):
        keep_str = "" if self.keepn is None else " | KEEP N = %s" % self.keepn
        return (
            "MNIST | ADAM, LR = %s | %s LOGIC | %s LINEAR CHANNELS | WIDTH %s LOGIC LAYER | %s LOGIC LAYERS | EXP %s%s"
            % ( 
                self.lr,
                self.get_logic_str().upper(), 
                self.lin_channels, 
                self.logic_width, 
                self.logic_layers, 
                self.exp,
                keep_str
                
            )
        ) 
        
    def regime_filename_elems(self):
        elems = [
            "mnist",
            str(self.lr),
            self.get_logic_str(),
            "%slc" % self.lin_channels,
            "%sw" % self.logic_width,
            "%slyrs" % self.logic_layers
        ]
        if self.exp != 1:
            elems.append("exp%s" % self.exp)
        if self.keepn is not None:
            elems.append("keep%s" % self.keepn)
        return elems
        
    def training_dataloader(self, run_no):
        return self.trainloader
    
    def testing_dataloader(self, run_no):
        return self.testloader
    
    def training_step(self, run_no, model):
        optim = self.get_optim(run_no)
        loss_fn = nn.CrossEntropyLoss()
        
        def step(tup):
            imgs, labels = tup
            preds = model(imgs.to(device))
            loss = loss_fn(preds, labels.to(device)) ** self.exp
            #losses.append(loss.item())

            optim.zero_grad(set_to_none=True)
            loss.backward()
            optim.step()
            
        return step
        
    def new_model(self):
        return EmbedMNIST(
            logic=self.flogic, 
            lin_channels=self.lin_channels, 
            logic_layers=self.logic_layers, 
            logic_width=self.logic_width,
        ).to(device)

In [12]:
class MNISTEmbedRegime(TrainingRegime):
    def __init__(self, lin_width=16, logic_width=64, logic_layers=1, embed_dims=3, reg_weight=0.01, lr=1e-2, exp=1, no_runs=1):
        super().__init__("./mnist/", no_runs)
        
        self.flogic = logic
        self.lin_width = lin_width
        self.logic_width = logic_width
        self.logic_layers = logic_layers
        self.embed_dims = embed_dims
        self.reg_weight = reg_weight
        self.lr = lr
        self.exp = exp
        
        self.tests = [
            CorrectOutputMetric(),
            LossMetric(),
            EmbedRegMetric(),
        ]
        
        self.trainloader = DataLoader(trainset, batch_size=128)
        self.testloader = DataLoader(testset, batch_size=128)
        
        self.optims = [None] * no_runs
        
    def get_optim(self, run_no):
        optim = self.optims[run_no - 1]
        if optim is None:
            model = self.get_loaded_model(run_no)
            optim = self.optims[run_no - 1] = (
                torch.optim.Adam(model.parameters(), lr=self.lr)
            )
        return optim
        
    def regime_str(self):
        return (
            "MNIST | ADAM, LR = %s | EMBEDDED LOGIC, %s DIMS | %s LOGIC REG WEIGHT | WIDTH %s LINEAR LAYER | WIDTH %s LOGIC LAYER | %s LOGIC LAYERS | EXP %s"
            % ( 
                self.lr,
                self.embed_dims,
                self.reg_weight,
                self.lin_width, 
                self.logic_width, 
                self.logic_layers, 
                self.exp,
                
            )
        ) 
        
    def regime_filename_elems(self):
        elems = [
            "mnist",
            str(self.lr),
            "embed",
            "%slw" % self.lin_width,
            "%sw" % self.logic_width,
            "%slyrs" % self.logic_layers,
            "%sdims" % self.embed_dims,
            "%sreg" % self.reg_weight,
        ]
        if self.exp != 1:
            elems.append("exp%s" % self.exp)
        return elems
        
    def training_dataloader(self, run_no):
        return self.trainloader
    
    def testing_dataloader(self, run_no):
        return self.testloader
    
    def training_step(self, run_no, model):
        optim = self.get_optim(run_no)
        loss_fn = nn.CrossEntropyLoss()
        
        def step(tup):
            imgs, labels = tup
            preds = model(imgs.to(device))
            loss = (loss_fn(preds, labels.to(device)) ** self.exp) #+ self.reg_weight * model.logic_reg()
            if self.reg_weight != 0:
                loss += self.reg_weight * model.logic_reg()
            
            optim.zero_grad(set_to_none=True)
            loss.backward()
            #grads = torch.tensor(0.0)
            #for param in model.parameters():
            #    with torch.no_grad():
            #        print(param.grad.abs().sum().cpu(), param.grad.shape)
            #raise Error
            #print(grads)
            optim.step()
            model.zero_reg()
            
        return step
        
    def new_model(self):
        return EmbedMNIST(
            lin_width=self.lin_width, 
            logic_layers=self.logic_layers, 
            logic_width=self.logic_width,
            embed_dims=self.embed_dims,
            reg=self.reg_weight != 0
        ).to(device)

In [13]:
def defer_real_regime(*args, **kargs):
    def return_regime():
        return MNISTRealRegime(*args, **kargs)
    return return_regime

def defer_embed_regime(*args, **kargs):
    def return_regime():
        return MNISTEmbedRegime(*args, **kargs)
    return return_regime

def defer_standard_regime(*args, **kargs):
    def return_regime():
        return MNISTRegime(*args, **kargs)
    return return_regime

real_regimes = [
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=32, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=32, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=64, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=64, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=32, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=32, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=64, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=64, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=32, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=32, logic_layers=2, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=64, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=64, logic_width=64, logic_layers=2, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=32, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=32, logic_layers=2, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=64, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=ProductLogic(), lin_channels=256, logic_width=64, logic_layers=2, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=32, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=32, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=64, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=64, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=32, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=32, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=64, logic_layers=1, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=64, logic_layers=2, keepn=None, lr=1e-2, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=32, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=32, logic_layers=2, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=64, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=64, logic_width=64, logic_layers=2, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=32, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=32, logic_layers=2, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=64, logic_layers=1, keepn=None, lr=1e-3, exp=8),
    defer_real_regime(logic=SchweizerSklarLogic(torch.tensor(-2.0).to(device)), lin_channels=256, logic_width=64, logic_layers=2, keepn=None, lr=1e-3, exp=8),
]

embed_regimes = [
    defer_embed_regime(lin_width=32, logic_width=32, logic_layers=2, embed_dims=5, reg_weight=0.000, lr=1e-2, exp=1, no_runs=1)
]

regimes = [
    #defer_standard_regime(lr=1e-3, no_runs=1),
    *embed_regimes
]

torch.autograd.set_detect_anomaly(True)
if True:
    regs = [regime_f() for regime_f in regimes]
    for reg in regs:
        reg.load_latest_models()
        reg.load_all_results()
    
    for i in range(0, 100):
        for reg in regs:
            #try:
                reg.loop_until(1, 1*(i+1))
            #except:
            #    pass

[MNIST | ADAM, LR = 0.01 | EMBEDDED LOGIC, 5 DIMS | 0.0 LOGIC REG WEIGHT | WIDTH 32 LINEAR LAYER | WIDTH 32 LO…

[MNIST | ADAM, LR = 0.01 | EMBEDDED LOGIC, 5 DIMS | 0.0 LOGIC REG WEIGHT | WIDTH 32 LINEAR LAYER | WIDTH 32 LO…

[MNIST | ADAM, LR = 0.01 | EMBEDDED LOGIC, 5 DIMS | 0.0 LOGIC REG WEIGHT | WIDTH 32 LINEAR LAYER | WIDTH 32 LO…

[MNIST | ADAM, LR = 0.01 | EMBEDDED LOGIC, 5 DIMS | 0.0 LOGIC REG WEIGHT | WIDTH 32 LINEAR LAYER | WIDTH 32 LO…

[MNIST | ADAM, LR = 0.01 | EMBEDDED LOGIC, 5 DIMS | 0.0 LOGIC REG WEIGHT | WIDTH 32 LINEAR LAYER | WIDTH 32 LO…

KeyboardInterrupt: 

In [ ]:
model = FuzzyMNIST(logic=ProductLogic(), conv_channels=16).cuda()

In [ ]:
def train_loop(model, optim, loader):
    loss_fn = nn.CrossEntropyLoss()
    losses = []
    
    for imgs, labels in loader:
        preds = model(imgs.cuda())
        loss = loss_fn(preds, labels.cuda())
        losses.append(loss.item())

        optim.zero_grad()
        loss.backward()
        optim.step()

    return torch.Tensor(losses)

In [ ]:
def test_loop(model, loader):
    correct_count = 0
    total_count = 0
    
    for imgs, labels in loader:
        preds = model(imgs.cuda())
        pred_labels = preds.argmax(dim=-1)
        correct_count += (pred_labels == labels.cuda()).sum()
        total_count += labels.numel()
        
    return correct_count / total_count

In [ ]:
def loop(model, optim, trainloader, testloader, epochs=1):
    losses = torch.zeros(0)
    test_accuracies = []
    
    for epoch_no in range(1, epochs+1):
        train_l = tqdm(trainloader, desc="Epoch #%s, Training" % epoch_no, leave=False)
        new_losses = train_loop(model, optim, train_l)
        losses = torch.cat((losses, new_losses), dim=0)
        
        test_l = tqdm(testloader, desc="Epoch #%s, Testing" % epoch_no, leave=False)
        test_accuracies.append(test_loop(model, test_l))
        
    return losses, torch.Tensor(test_accuracies)

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
losses, accuracies = loop(model, optim, trainloader, testloader, epochs=500)

In [ ]:
test_loop(model, testloader)

In [ ]:
torch.rand(3,4).argmax(dim=-1)

In [ ]:
accuracies

In [ ]:
"-".join(["0","1"])

In [ ]:
[0,1] == [1,0]

In [ ]:
flatten([(0,1),(2,3)])

In [ ]:
regs[0].results

In [ ]:
regs[0].models[0].to(device)

In [ ]:
torch.arange(27).reshape(3,3,3)[(*((slice(None,None,None),)*0),torch.randperm(3))]

NameError: name 'SchweizerSklarLogic' is not defined